In [1]:
from __future__ import print_function
import h5py
import numpy as np
from tqdm import tqdm

import torch.utils.data
from torch import nn, optim
from torchinfo import summary

/data1/jcampos/.conda/envs/ml4qick-env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data1/jcampos/.conda/envs/ml4qick-env/lib/python3.7/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
from base_model import TinyClassifier 
from hawq_model import HawqTinyClassifier 
from brevitas_model import BrevitasTinyClassifier 

In [3]:
from brevitas.export.onnx.qonnx.manager import QONNXManager
from hawq.utils.export import ExportManager

### Setup QAT training parameters

In [4]:
framework = "hawq"  # options: hawq/brevitas

# Hyper-Parameters
torch.manual_seed(4)
epochs = 210
train = 9000#data.shape[0]*0.99
batch_size = 12800
learning_rate = 1e-4

device = 'cuda' if torch.cuda.is_available() else 'cpu'
kwargs = {'num_workers':0, 'pin_memory': True} 
plot_interval = 1

In [5]:
if framework.lower() == "hawq":
    base_model = TinyClassifier()
    base_model.load_state_dict(torch.load("../../checkpoints/checkpoint_tiny_affine.pth"))
    model = HawqTinyClassifier(base_model)
elif framework.lower() == "brevitas":
    model = BrevitasTinyClassifier()

In [6]:
# Load the raw data
IF = -136.75/1e3
with h5py.File(r'../../../datasets/qubits/00002_IQ_plot_raw.h5', 'r') as f:
    adc_g_1 = np.array(f['adc_g_1'])[0]
    adc_g_2 = np.array(f['adc_g_2'])[0]
    adc_e_1 = np.array(f['adc_e_1'])[0]
    adc_e_2 = np.array(f['adc_e_2'])[0]

""" Select the range of time series data. Each data is 2000 element vector 
representing 2000ns readout signal"""
csr = range(500,1500)
sr = len(csr)

I_g = adc_g_1[:,csr]
Q_g = adc_g_2[:,csr] 
I_e = adc_e_1[:,csr] 
Q_e = adc_e_2[:,csr] 

# Dataset Creation
data = np.zeros((adc_g_1.shape[0]*2,sr,2))
data[0:adc_g_1.shape[0],:,0] = I_g
data[0:adc_g_1.shape[0],:,1] = Q_g
data[adc_g_1.shape[0]:adc_g_1.shape[0]*2,:,0] = I_e
data[adc_g_1.shape[0]:adc_g_1.shape[0]*2,:,1] = Q_e

labels = np.zeros(I_e.shape[0]*2)
labels[I_e.shape[0]:I_e.shape[0]*2] = 1

data = torch.from_numpy(data).float()
labels = torch.from_numpy(labels).float()

class Qubit_Readout_Dataset():
    
    def __init__(self):
        self.data = data
        self.labels = labels    
        self.data = self.data.reshape(len(data),sr*2)
       
    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


# Dataloader Prep
test = len(data)-train

dataset = Qubit_Readout_Dataset()
train_data, test_data = torch.utils.data.random_split(dataset, [int(train), int(test)])

num_workers = 0
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, 
                                            num_workers = num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, 
                                            num_workers = num_workers, shuffle=True)


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

# Loss tracks
train_loss_track=np.array([])
test_loss_track=np.array([])
acc_track=np.array([])

In [8]:
# Training loop
for epoch in tqdm(range(epochs)):
    
    train_loss = 0
    model.train() 
    
    for data, labels in train_loader:
        optimizer.zero_grad()
        data, labels = data.to(device), labels.to(device)
        states = model(data)
        loss = criterion(states, labels.long())
        loss.backward()
        train_loss += loss.detach().cpu().numpy()
        optimizer.step()
        
    train_loss_track = np.append(train_loss_track,np.asarray(train_loss))
        
    test_loss = 0    
    model.eval()
    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            print(data.shape)
            states = model(data)
            loss = criterion(states, labels.long())
            test_loss += loss.detach().cpu().numpy()
    
    test_loss_track = np.append(test_loss_track,np.asarray(test_loss))
    
    if epoch % plot_interval == 0:
        print('====> Epoch: {} Training loss: {:.6f}'.format(
                  epoch, train_loss ))
        print('====> Epoch: {} Test loss: {:.6f}'.format(
                  epoch, test_loss ))

        p1, p2 = 0, 800

    # Readout Fidelity
    model.eval() 
    cc = 0
    y_true = torch.tensor([]).to(device)
    y_pred = torch.tensor([]).to(device)
    
    with torch.no_grad():
        for data, target in test_loader:
            
            data=data.to(device)
            states = model(data)
            target = target.to(device) 
    
            val, ind = torch.max(states,1)
            y_pred = torch.cat((y_pred, ind), 0)
            y_true = torch.cat((y_true, target), 0)
    
    acc = y_true-y_pred
    accuracy = (len(y_true)-torch.count_nonzero(acc))/len(y_true)
    accuracy = accuracy.item()
    
    acc_track = np.append(acc_track,np.asarray(accuracy))
    print('Readout Fidelity: %', accuracy*100)


  0%|          | 1/210 [00:00<01:04,  3.25it/s]

torch.Size([1000, 2000])
====> Epoch: 0 Training loss: 0.108649
====> Epoch: 0 Test loss: 0.287474
Readout Fidelity: % 90.6000018119812


  1%|          | 2/210 [00:00<01:05,  3.20it/s]

torch.Size([1000, 2000])
====> Epoch: 1 Training loss: 0.105176
====> Epoch: 1 Test loss: 0.219233
Readout Fidelity: % 90.89999794960022


  1%|▏         | 3/210 [00:00<00:58,  3.56it/s]

...

torch.Size([1000, 2000])
====> Epoch: 207 Training loss: 0.058208
====> Epoch: 207 Test loss: 0.252980
Readout Fidelity: % 91.00000262260437


100%|█████████▉| 209/210 [01:01<00:00,  3.01it/s]

torch.Size([1000, 2000])
====> Epoch: 208 Training loss: 0.058177
====> Epoch: 208 Test loss: 0.253342
Readout Fidelity: % 91.00000262260437


100%|██████████| 210/210 [01:01<00:00,  3.41it/s]

torch.Size([1000, 2000])
====> Epoch: 209 Training loss: 0.058148
====> Epoch: 209 Test loss: 0.253251
Readout Fidelity: % 91.00000262260437


In [9]:
torch.save(model.state_dict(), f"checkpoints/{framework}_ckp.pth")

In [10]:
if framework.lower() == "hawq":
    manager = ExportManager(model)
    manager.export(torch.randn(1, 2000), "checkpoints/hawq.onnx")
elif framework.lower() == "brevitas":
    QONNXManager.export(model, input_shape=(1, 2000), export_path='checkpoints/brevitas.onnx')

Exporting model...
Optimizing...
Model saved to: checkpoints/hawq.onnx
